# Introduction
This notebook will fetch data from an external API, process them, and load them to S3 bucket as a staging area.

In [1]:
import requests
import pandas as pd
from datetime import datetime
import json
import configparser
import boto3
import os

In [3]:
config = configparser.ConfigParser()
config.read("../config.cfg")

# retrieve access key and secret key
aws_access_key_id = config['AWS']['aws_access_key_id']
aws_secret_access_key = config['AWS']['aws_secret_access_key']

## Fetch quick data from RapidAPI

In [4]:
def get_stock_data(symbol, rangeData = "1mo", interval = "1h"):
    base_url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/market/get-charts"
    querystring = {
        "region":"US", 
        "lang":"en", 
        "symbol": symbol, 
        "range": rangeData, 
        "interval": interval
    }
    headers = {
        'x-rapidapi-host': config["RapidAPI"]['x-rapidapi-host'],
        'x-rapidapi-key': config['RapidAPI']['x-rapidapi-key'],
        'Content-Type': "application/json"
    }
    response = requests.request("GET", base_url, headers=headers, params=querystring)
    if response.ok:
        data = json.loads(response.content)
        return data['chart']['result'][0]
    return None

In [5]:
response = get_stock_data("AAPL")
response

## Transform raw json data to Dataframes

In [7]:
metadata = response['meta']
timestamps = response['timestamp']
indicators = response['indicators']['quote'][0]

In [14]:
table = {
    "Timestamps": timestamps, 
    "Volume": indicators['volume'], 
    "Low": indicators["low"],
    "Open": indicators["open"], 
    "High": indicators["high"], 
    "Close": indicators["close"]
}
df_indicators = pd.DataFrame(table)
df_indicators['Datetime'] = (df_indicators['Timestamps']
                             .apply(lambda x: datetime.fromtimestamp(int(x)).strftime("%Y-%m-%d %H:%M:%S")))
df_indicators['symbol'] = metadata['symbol']
df_indicators.head()

In [17]:
def get_trading_period(trade_period):
    trading_periods = set([])
    for period in trade_period:
        start = datetime.fromtimestamp(int(period[0]['start'])).strftime("%H:%M:%S")
        end = datetime.fromtimestamp(int(period[0]['end'])).strftime("%H:%M:%S")
        date = f"{start} - {end}"
        trading_periods.add(date)
    return list(trading_periods)

impt_metadata = {
    "currency": metadata['currency'],
    "symbol": metadata['symbol'],
    "instrumentType": metadata['instrumentType'],
    "firstTradeDate": datetime.fromtimestamp(metadata['firstTradeDate']).strftime("%Y-%m-%d %H:%M:%S"),
    "exchangeTimezoneName": metadata["exchangeTimezoneName"],
    'timezone': metadata['timezone'],
    'trade_period': get_trading_period(metadata['tradingPeriods']),
    'range': metadata['range'],
    'interval': metadata['dataGranularity'],
    'start_date': df_indicators['Timestamps'].min()
}

df_meta = pd.DataFrame(impt_metadata)
df_meta.head()

## Upload CSV files to S3 bucket

In [10]:
# instantiate s3 objects
s3 = boto3.resource("s3", 
                    region_name = 'us-west-2', 
                    aws_access_key_id = aws_access_key_id,
                    aws_secret_access_key = aws_secret_access_key
                   )

# function to upload local file object to S3 bucket
def upload_object(s3, local_path, tag, obj, bucket, keys, ext="csv"):
    file_name = str(keys['start_date']) + f".{ext}"
    directory = os.path.join(local_path, keys['symbol'], tag, keys['range'], keys['interval'])
    file_path = os.path.join(directory, file_name)
    
    # upload file locally
    if ext == "csv":
        if not os.path.exists(directory): 
            os.makedirs(directory)
        obj.to_csv(file_path, header=False, index=False)
    elif ext == "json":
        with open(file_path, 'w') as file:
            json.dump(obj, file)
    else:
        print("Invalid file extension")
        return
    
    # upload file to s3
    key = os.path.join(keys['symbol'], tag, keys['range'], keys['interval'], file_name)
    try:
        s3.Bucket(bucket).upload_file(file_path, key)
        print(f"Successfully uploaded an object @ {local_path} to S3 @ s3://{bucket}/{key}")
    except Exception as e:
        print(e)
        
    return os.path.join(keys['symbol'], tag, keys['range'], keys['interval'])

In [11]:
# retrieve bucket_name from config object
bucket_name = config['S3']['bucket_name']

keys = dict(df_meta.iloc[0, :])

# upload metadata dataframe
key_path_meta = upload_object(
    s3 = s3,
    local_path = "../data/s3",
    tag = 'metadata',
    obj = df_meta,
    bucket = bucket_name,
    keys = keys,
    ext = "csv"
)    

# upload indicators dataframe
key_path_indicators = upload_object(
    s3 = s3,
    local_path = "../data/s3",
    tag = 'indicators',
    obj = df_indicators,
    bucket = bucket_name,
    keys = keys,
    ext = "csv"
)

Successfully uploaded an object @ ../data/s3 to S3 @ s3://nyse-stock-data-dngo/AAPL/metadata/1mo/5m/1635256800.csv
Successfully uploaded an object @ ../data/s3 to S3 @ s3://nyse-stock-data-dngo/AAPL/indicators/1mo/5m/1635256800.csv


### Store data to share among notebooks

In [64]:
%store key_path_meta
%store key_path_indicators

Stored 'key_path_meta' (str)
Stored 'key_path_indicators' (str)
